In [1]:
from config import VK_TOKEN

import tqdm
import json
import vk_api
import re
from fuzzywuzzy import process

In [2]:
vk_session = vk_api.VkApi(token=VK_TOKEN, app_id=5820017)
vk_session._auth_token()

vk = vk_session.get_api()

In [3]:
tools = vk_api.VkTools(vk_session)

In [10]:
vegan_recipes = tools.get_all_iter("wall.search", 20, 
                                   values={"owner_id": -35838585, "query": "#vegan", "owners_only": True})

In [4]:
# downloaded_vegan_recipes = []
# for recipe in tqdm.tqdm(vegan_recipes):
#     downloaded_vegan_recipes.append(recipe)
# json.dump(downloaded_vegan_recipes, open("assets/downloaded_vegan_recipes.json", "w"))
downloaded_vegan_recipes = json.load(open("assets/downloaded_vegan_recipes.json"))

In [74]:
def process_recipe(recipe):
    tokens = recipe['text'].lower().split()
    if not tokens:
        return None
    # best occurence of fuzzed ингредиент word
    ingr_word = process.extractOne("ингредиент", tokens)[0]
    ingr_idx = recipe['text'].lower().index(ingr_word)
    # best occurence of fuzzed приготовление word
    prigot_word = process.extractOne("приготовление", tokens)[0]
    prigot_idx = recipe['text'].lower().index(prigot_word)
    
    name = recipe['text'][:ingr_idx].strip()
    name, *description = name.rsplit("\n")
    description = "\n".join(description).strip()
    ingredients = recipe['text'][ingr_idx+len(ingr_word):prigot_idx].split("\n")
    ingredients = [it.strip() for it in ingredients if it.strip()]
    script = recipe['text'][prigot_idx+len(prigot_word):].split("\n")
    script = [it.strip() for it in script if it.strip()]
    return name, description, ingredients, script

In [65]:
for recipe in downloaded_vegan_recipes[5:6]:
    name, description, ingredients, script = process_recipe(recipe)
    print("название", name)
    print("описание", description)
    print("игредиенты", ingredients)
    print("приготовление", script)

название ФРУКТОВЫЙ ДЕСЕРТ НА АГАР-АГАРЕ
описание 
игредиенты ['• 400 мл сока/компота/морса', '• 1 стакан черной смородины (клубника, голубика, черника)', '• 1 стакан черешни/вишни без косточек', '• 1 небольшое яблоко', '• 2 ч. л. агар-агара']
приготовление ['1. Смешайте агар-агар с 200 мл сока, дайте постоять 5 минут.', '2. Нарежьте фрукты и уложите их вместе с ягодами в глубокую прямоугольную форму слоями. Нижний слой будет верхушкой десерта. Старайтесь оставлять место между ягодами, чтобы было потом куда выливать желе. Если форма силиконовая - оставьте как есть, если керамическая или стеклянная - проложите пищевой пленкой, чтобы было удобнее потом извлекать готовый десерт.', '3. Поставьте на огонь смесь сока и агар-агара, доведите до кипения, влейте остаток сока и дайте покипеть 2–3 минуты. Периодически мешайте смесь, чтобы агар-агар растворился.', '4. Снимите с огня, дайте остыть буквально 5 минут и залейте ягоды. Как правило, желе застывает само при комнатной температуре, но лучше 

In [66]:
different_measures = {
    "щеп(отк(а|у|и))?": 2,
    "ч\.\s?л": 5,
    "ст\.\s?л": 15,
    "((ч(айн)?|ст)(олов)?\.?(ые|ых|ая|ой)?)?\s?ложе?к(а|и)?": 5,
    "гр?(амм)?": 1, 
    "зуб(к|чик)?\.?(а|ов)?": 5,
    "головк(а|и)": 5,
    "шт(ук(а|и)?)?": None,
    "кусоч\.?(ка|ек)?": None,
    "веточк(а|и)": 2,
    "кружк(а|и)?": 200,
    "чашк(а|и)?": 200,
    "ст(ак)?\.?(ан|ана)?": 200,
    "банк(а|и)?": 300,
    "ломтик(а|и|ов)?": None,
    "стеб(л(я|ей|евой)|ель(ка)?|)": None,
    "струч(ок|к(а|ов))": None,
    "пуч(ок)?": 50,
    "килограмм": 1000,
    "кг": 1000,
    "по( |\-)вкусу": 10,
    "лист(ы|ья|а|ьев|ов|ик(а|ов)?|очк(а|ов|и)?)": 3, 
    "с?м": None,
    "коч(ан)?": 1000,
    "м?л": None
}
digit_pattern = "\d+((,|\.)?\d*)?"
number_pattern = f"(\d*\s*(½|¼|⅔|⅛|⅓)|{digit_pattern}(\/|\\|\\\\|-)?({digit_pattern})*)"  # group1
measures_pattern = f"({'|'.join(different_measures)})"

In [67]:
re_pattern = re.compile(f"({number_pattern}\s*{measures_pattern}+\.?(?!\S)|{measures_pattern}\.?)")

In [68]:
def parse_number(str_num):
    if "-" in str_num:
        str_num = str_num.split("-")[-1]
    str_num = str_num.replace(",", ".").replace(" ", "")
    str_num = str_num.replace("\\", "/").replace("\\\\", "/")
    str_num = str_num.replace("½", "+0.5").replace("¼", "+0.25").replace("⅔", "+0.66").replace("⅛", "+0.13").replace("⅓", "+0.33")
    return eval(str_num)

In [69]:
def parse_measure(str_measure, different_measures):
    group_measures_pattern = "|".join([f"({p})" for p in different_measures])
    for p, v in different_measures.items():
        match = re.search(p, str_measure)
        if match:
            return v

In [70]:
def get_ingr_re_matches(ingr):
    ingr = ingr.replace(";", " ").replace(":", " ").replace(")", " ").replace("(", " ").lower().strip()
    if ingr and ingr[-1] == ",":
        ingr = ingr[:-1]
    ingr_cp = ingr
    split_word_indices = []
    prev_index = 0
    for w in ingr.split():
        ind = ingr_cp.index(w, prev_index)
        split_word_indices.append((ind-1, ind+len(w)))
        prev_index = ind+len(w)
    ingr_matches = []
    matches = re.finditer(re_pattern, ingr)
    for m in matches:
        indices = m.span()
        for split_word_ind in split_word_indices:
            if split_word_ind[1] > indices[1] and split_word_ind[0] < indices[0]:
                break
        else:
            ingr_matches.append(indices)
    return ingr_matches

In [79]:
def clean_ingr_str(ingr, number_pattern):
    """
    Would work if initial search failed
    """
    food_ingr_match = re.search("[а-яА-Я\s]+", ingr)
    number_match = re.search(number_pattern, ingr)
    if food_ingr_match:
        ingr = food_ingr_match.group()
    ingr = ingr.strip()
    if number_match:
        number = parse_number(number_match.group())
        if len(ingr):
            return (ingr, number)
    else:
        if len(ingr):
            return (ingr, 1)

In [82]:
def parse_ingr_matches(ingr_matches, number_pattern, different_measures):
    """
    Would work if matches were found – they are indices
    For each set of indices inject number and other substring
    Normalize measure string into grams, get only cyrillic food string
    """
    parsed_ingr_matches = {}
    for ind in ingr_matches:
        ingr_match = ingr[ind[0]:ind[1]]
        number_ind = re.match(number_pattern, ingr_match)
        if not number_ind:
            continue
        number_ind = number_ind.span()
        number = ingr_match[number_ind[0]:number_ind[1]].strip()
        number = parse_number(number)
        measure = ingr_match[:number_ind[0]] + ingr_match[number_ind[1]:].strip()
        measure_grams = parse_measure(measure, different_measures)
        if measure_grams:
            total_grams = number * measure_grams
        else:
            total_grams = (number, )
        food_ingr = ingr[:ind[0]] + ingr[ind[1]:]
        food_ingr_match = re.search("[а-яА-Я\s]+", food_ingr)
        if food_ingr_match:
            food_ingr = food_ingr_match.group()
        food_ingr = food_ingr.strip()
        if len(food_ingr):
            parsed_ingr_matches[food_ingr] = total_grams
    return parsed_ingr_matches

In [89]:
n_recipes = 0
n_success = 0
for recipe in downloaded_vegan_recipes[:5]:
    processed = process_recipe(recipe)
    if not processed:
        continue
    name, description, ingredients, script = processed
    processed_ingredients = []
    for ingr in ingredients:
        ingr_matches = get_ingr_re_matches(ingr)
        if not len(ingr_matches):
            ingr = clean_ingr_str(ingr, number_pattern)
            if ingr:
                processed_ingredients.append(ingr)
        else:
            parsed_ingr_matches = parse_ingr_matches(ingr_matches, number_pattern, different_measures)
            for k, v in parsed_ingr_matches.items():
                if isinstance(v, tuple):
                    processed_ingredients.append((k, v[0]))
                else:
                    processed_ingredients.append({k: v})
    if name and processed_ingredients:
        print(name, processed_ingredients)
        n_success += 1
    else:
        pass
    n_recipes += 1
print(f"{n_success/n_recipes:.2%} successful matches")

Вареники с гречкой и картофелем [('Для теста', 1), {'отбеленной пшеничной муки': 400}, {'цельнозерновой муки': 150}, ('кипятка', 450), {'соли': 5}, {'растительного масла': 50.0}, ('Для начинки', 1), {'гречневой крупы': 200}, {'картофеля': 800}, {'растительного масла': 50.0}, ('Для подачи', 1), {'растительного масла': 50.0}, ('консервированные овощи', 1)]
Банановые ромбики [('Банан', 1), {'Мука': 800}, {'Мед': 30}, {'Сахар': 200}, {'Сода': 5}]
Испанский омлет [{'оливкового масла': 10}, {'картофеля': 300}, {'нутовой муки': 200}, {'воды': 200}]
Паста в соусе с баклажаном [('на', 4), {'пасты': 300.0}, ('баклажана', 1), {'нарезанных шампиньонов': 400}, {'соуса маринара': 300.0}, {'воды': 400}, {'морской соли': 10}, {'молотого ч': 5}, {'оливкового масла': 30}, ('на выбор', 1)]
Вэрзэре — пироги с капустой  [('Тесто', 1), ('Вода', 100), ('Масло растительное', 100), {'Соль': 2.5}, {'Мука': 400}, ('Начинка', 1), {'Капуста квашеная': 300}, ('Лук репчатый', 1), ('Лук зеленый', 3)]
100.00% successf